In [27]:
from datetime import datetime
import local_helpers as lh
import vaex
import h3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
from shapely import wkt
from sklearn.preprocessing import TargetEncoder

# Feature Engineering

### Previous demand as input

As we have given time series data, it is a common approach to use the demand of previous hours (or days etc.) as an input for the prediction. The assumption we hereby make is that the factors that influence the demand have not changed dramatically within the used time frames. We have decided to construct the following features from previous demand:

* 2 hour: The asssumption is that the demand should not change dramatically between three hours.
* 24 hours: The asssumption is that the current demand should be comparable to the demand exactly one day ago, as factors such as season, time of the day are the same.
* Average demand of the past week at the same day time: This feature is the average of all 7 demand observations of the past week at same time of the day. 

In [5]:
# This functions loads the dataset either with hexagons or census tract
def load_dataset_to_pandas(resolution=10, location_type_hexagon = True):
    df = vaex.open('./data/trips_prepared.hdf5')
    df.head()

    df["trip_start_day"] = df.trip_start_timestamp.dt.day
    df["trip_start_month"] = df.trip_start_timestamp.dt.month
    df["trip_start_hour"] = df.trip_start_timestamp.dt.hour
    df["trip_start_minute"] = df.trip_start_timestamp.dt.minute

    if location_type_hexagon:
        
        
    
        RESOLUTION = resolution
        def geo_to_h3(row1, row2):
            return h3.geo_to_h3(row1,row2, RESOLUTION)

        # Step 1: For each pickup and drop-off calculate the correct hexagon in the resolution
        df['pickup_loc'] = df.apply(geo_to_h3, [df['pickup_centroid_latitude'], df['pickup_centroid_longitude']])
    else:
        df.rename('pickup_census_tract', 'pickup_loc')

    ### Group by hour
    df_demand = df.groupby(['trip_start_hour', 'trip_start_month', 'trip_start_day', 'pickup_loc']).agg({'demand': 'count'})
    # Add timestamp as preparation for resampling
    df_demand['timestamp'] = pd.to_datetime({'year': 2017, 'month': df_demand['trip_start_month'].to_numpy(), 'day': df_demand['trip_start_day'].to_numpy(), 'hour': df_demand['trip_start_hour'].to_numpy()}).to_numpy()

    # convert to pandas df
    df_demand = df_demand.to_pandas_df()
    return df_demand


In [6]:
def resample_to_hourly(df):
    ### Creation of dummy df which contains hourly data dummy data over an entire year per hexagon
    # Create a DateTimeIndex with hourly intervals for the year 2017
    start_date = '2017-01-01 00:00:00'
    end_date = '2017-12-31 23:00:00'
    hourly_range = pd.date_range(start=start_date, end=end_date, freq='H')
    num_entries_per_year = len(hourly_range)
    hour_range = np.tile(hourly_range.hour,len(np.unique(df.pickup_loc)))
    month_range = np.tile(hourly_range.month,len(np.unique(df.pickup_loc)))
    day_range = np.tile(hourly_range.day,len(np.unique(df.pickup_loc)))
    hourly_range = np.tile(hourly_range,len(np.unique(df.pickup_loc)))

    # -1 values will indacte that these rows were artificially generated later on
    data = {
        'trip_start_hour': hour_range,
        'trip_start_month': month_range,
        'trip_start_day': day_range,
        'pickup_loc': np.repeat(np.unique(df.pickup_loc), num_entries_per_year),
        'demand': 0,
    }

    df_hourly = pd.DataFrame(data, index=hourly_range)
    df_hourly= df_hourly.set_index([df_hourly.index, 'pickup_loc'])

    # introduce multiindex for filling up the df with hourly index later on
    df=df.set_index(['timestamp', 'pickup_loc'])

    # insert df 
    df_hourly.update(df)

    # clear up multi-index
    df_hourly=df_hourly.reset_index()
    df_hourly.columns = ['timestamp','pickup_loc','trip_start_hour','trip_start_month','trip_start_day','demand']
    return df_hourly
    

In [7]:
def get_target_time_bucket_demand(df):
    
    demand_cols = [f'demand_n-{num}' for num in list(range(0,25))]
    
    for shift in list(range(0,25)):
        df[f'demand_n-{shift}'] =  df.sort_values('timestamp').groupby('pickup_loc')['demand'].shift((-1)*shift)
        
    two_hours = ['demand_n-0','demand_n-1']
    six_hours = [f'demand_n-{num}' for num in list(range(0,6))]
    twentyfour_hours = [f'demand_n-{num}' for num in list(range(0,24))]
    
    df['demand_target_bucket_1'] = df['demand_n-0']
    df['demand_target_bucket_2'] = df[two_hours].sum(axis=1)
    df['demand_target_bucket_6'] = df[six_hours].sum(axis=1)
    df['demand_target_bucket_24'] = df[twentyfour_hours].sum(axis=1)
    df = df.drop(labels=demand_cols, axis=1)
       
    
    return df

In [8]:
def get_past_time_bucket_demand(df):
    
    demand_cols = [f'demand_p-{num}' for num in list(range(0,24))]
    
    for shift in list(range(0,25)):
        df[f'demand_p-{shift}'] =  df.sort_values('timestamp').groupby('pickup_loc')['demand'].shift(shift)
        
    two_hours = ['demand_p-24', 'demand_p-23']
    six_hours = [f'demand_p-{num}' for num in list(range((24-5),25))]
    twentyfour_hours = [f'demand_p-{num}' for num in list(range(1,25))]
    
    df['demand_bucket_2'] = df[two_hours].sum(axis=1)
    df['demand_bucket_6'] = df[six_hours].sum(axis=1)
    df['demand_bucket_24'] = df[twentyfour_hours].sum(axis=1)
    df = df.drop(labels=demand_cols, axis=1)
    
    
    
    return df

In [9]:
def get_past_single_hour_demand(df):
    # insert features 1, 2 and 24 hours previous demand
    df['demand_h-1'] = df.sort_values('timestamp').groupby('pickup_loc')['demand'].shift(1)
    df['demand_h-2'] = df.sort_values('timestamp').groupby('pickup_loc')['demand'].shift(2)
    df['demand_h-24'] = df.sort_values('timestamp').groupby('pickup_loc')['demand'].shift(24)
    return df

# Weather features
In the descriptive analysis, particularly the analysis of temporal demand patterns, we found that the temperature and demand curves follow similar directions. Therefore, we construct features based on temperature to enable models that capture this relationship.

### Include weather data
First, we have to include the weather data into the dataframe. For this we just need to merge the two datasets, as both are already in hourly frequency. The weather data propose data for minute 53 of an hour. Therefore, we round up to the nearest hour for each row. We suppose that the weather changes in seven minutes can be disregarded.

In [22]:
def weather_categories(weather):
    if "Fair" in weather:
        return "Clear/Sunny"
    elif "Cloudy" in weather:
        return "Cloudy"
    elif "Rain" in weather or "Drizzle" in weather or "T-Storm" in weather or "Thunder" in weather:
        return "Rain/Storms"
    elif "Snow" in weather or "Wintry" in weather:
        return "Snow/Winter Conditions"
    elif weather in ["Fog", "Haze", "Smoke", "Widespread Dust / Windy"]:
        return "Other"
    else:
        return "Miscellaneous"
    
def merge_weather(df):
    # read and merge weather data
    df_weather = pd.read_csv('data/weather_data_final.csv')
    df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])
    df_weather['date_time'] = df_weather['date_time'].dt.ceil('H')
    df_weather.rename(columns={'date_time': 'timestamp'}, inplace=True)

    # cast data types
    df_weather['temp'] = df_weather['temp'].str.replace('\xa0°F', '').astype(float).fillna(np.nan)
    df_weather['dew_point'] = df_weather['dew_point'].str.replace('\xa0°F', '').astype(float).fillna(np.nan)
    df_weather['humidity'] = df_weather['humidity'].str.replace('\xa0°%', '').astype(float).fillna(np.nan)
    df_weather['wind_speed'] = df_weather['wind_speed'].str.replace('\xa0°mph', '').astype(float).fillna(np.nan)
    df_weather['wind_gust'] = df_weather['wind_gust'].str.replace('\xa0°mph', '').astype(float).fillna(np.nan)
    df_weather['pressure'] = df_weather['pressure'].str.replace('\xa0°in', '').astype(float).fillna(np.nan)
    df_weather['precip'] = df_weather['precip'].str.replace('\xa0°in', '').astype(float).fillna(np.nan)
    df_weather = df_weather.drop(['time','date', 'dew_point'], axis=1)
    df_weather['condition'] = df_weather["condition"].apply(weather_categories)
    
    dummies = pd.get_dummies(df_weather['condition']).astype(int)
    df_weather = pd.concat([df_weather.drop(['condition'], axis=1), dummies], axis=1)
    df = df.merge(df_weather, on='timestamp', how='left')
    return df

### Temperature features
In addition to the current temperature, we are add the temperature from 1, 2, and 3 hours prior to the time of taxi demand. We suggest that past temperature conditions could potentially impact the decision to hire a taxi.

In [11]:
def get_past_temperature(df):
    df['temp_h-1'] = df.sort_values('timestamp').groupby('pickup_loc')['temp'].shift(1)
    df['temp_h-2'] = df.sort_values('timestamp').groupby('pickup_loc')['temp'].shift(2)
    df['temp_h-3'] = df.sort_values('timestamp').groupby('pickup_loc')['temp'].shift(3)
    return df

### Precipitation
We hypothesize that precipitation has a significant impact on demand. Therefore, we construct features that describe whether it has rained in the last 1-3 hours.

In [12]:
def get_past_precip(df):
    df['precip_h-1'] = df.sort_values('timestamp').groupby('pickup_loc')['precip'].shift(1)
    df['precip_h-2'] = df.sort_values('timestamp').groupby('pickup_loc')['precip'].shift(2)
    df['precip_h-3'] = df.sort_values('timestamp').groupby('pickup_loc')['precip'].shift(3)
    return df

### Season
We suggest that demand changes over seasons.

In [39]:
def date_to_season(date):
    if pd.Timestamp(2017, 12, 22) <= date or date < pd.Timestamp(2017, 3, 20):
        return 'winter'
    elif pd.Timestamp(2017, 3, 20) <= date < pd.Timestamp(2017, 6, 21):
        return 'spring'
    elif pd.Timestamp(2017, 6, 21) <= date < pd.Timestamp(2017, 9, 23):
        return 'summer'
    else:
        return 'autumn'

def get_season(df):
    # Vectorize the date_to_season function
    date_to_season_vec = np.vectorize(date_to_season)
    
    # Apply the vectorized function to the 'timestamp' column
    df['season'] = date_to_season_vec(df['timestamp'])
    
    # Create dummies
    dummies = pd.get_dummies(df['season']).astype(int)
    df = pd.concat([df.drop(['season'], axis=1), dummies], axis=1)    
    return df

### Weekend feature
In the descriptive analysis we have seen that demand changes depending on weekend or not. Hence we engineer a feature "weekend" which is 1 for all rides on saturday & sunday and zero for all other days.

In [14]:
def get_weekend(df):
    df['weekend'] = (df['timestamp'].dt.weekday >= 5).astype(int)
    return df

### Daytime features
In addition, descriptive analysis has shown that the time of day, i.e., whether it is night, morning, afternoon, or evening, plays an important role in determining demand. Therefore, we developed four characteristics, each indicating whether a trip occurs during the following times.
* Morning: 6 a.m. - 12 p.m.
* Afternoon: 12 noon - 6 p.m.
* Evening: 6 p.m. - 11 p.m.
* Night: 12 a.m. - 6 a.m.

In [15]:
def get_pnt_day_with_pnt_week(dt):
    dt = pd.to_datetime(dt)  
    time_of_week = "week" if dt.weekday() < 5 else "weekend"
    hour = dt.hour

    if 6 <= hour < 12:
        return "morning_" + time_of_week
    elif 12 <= hour < 18:
        return "afternoon_" + time_of_week
    elif 18 <= hour < 23:
        return "evening_" + time_of_week
    else:
        return "night_" + time_of_week

def get_daytime(df):
    vfunc = np.vectorize(get_pnt_day_with_pnt_week)
    df['daytime'] = vfunc(df['timestamp'])
    dummies = pd.get_dummies(df['daytime']).astype(int)
    df = pd.concat([df.drop(['daytime'], axis=1), dummies], axis=1)
    return df


### Event Features
Events such as public holidays might influence the demand. In order to capture these patterns we introduce the public holiday feature

There were several public holidays in Boston:
* Martin Luther King Day: Monday, January 16, 2017
* Lincoln's Birthday: Monday, February 13, 2017
* Washington's Birthday (President's Day): Monday, February 20, 2017
* Memorial Day: Monday, May 29, 2017
* Independence Day: Tuesday, July 04, 2017
* Labor Day: Monday, September 04, 2017
* Columbus Day: Monday, October 09, 2017
* Veterans' Day: Friday, November 10, 2017
* Thanksgiving Day: Thursday, November 23, 2017
* Thanksgiving Day: Friday, November 24, 2017
* Christmas Day: Monday, December 25, 2017

These events might have influenced the demand

In [16]:
holiday_dates = [
    datetime(2017, 1, 2),   # New Year's Day
    datetime(2017, 1, 16),  # Martin Luther King Day
    datetime(2017, 2, 13),  # Lincoln's Birthday
    datetime(2017, 2, 20),  # Washington's Birthday (President's Day)
    datetime(2017, 5, 29),  # Memorial Day
    datetime(2017, 7, 4),   # Independence Day
    datetime(2017, 9, 4),   # Labor Day
    datetime(2017, 10, 9),  # Columbus Day
    datetime(2017, 11, 10), # Veterans' Day
    datetime(2017, 11, 23), # Thanksgiving Day
    datetime(2017, 11, 24), # Day after Thanksgiving
    datetime(2017, 12, 25), # Christmas Day
]

def get_holiday_dates(df):
    df['is_holiday'] = df['timestamp'].isin(holiday_dates).astype(int)
    return df

### Neighbor demand feature
We expect a high correlation between the demand of one hexagon and the demand in the surrounding hexagons. With this feature we can observe demand patterns in a greater radius than only in the observed location.

In [17]:
def get_neighbor_demand(df):

    if h3.h3_is_valid(df["pickup_loc"].iloc[0]):
        pickup_locs = df["pickup_loc"].unique()
        neighbors_dict = {}
        for loc in pickup_locs:
            neighbors_dict[loc] = h3.k_ring(loc, k=1)
    else:
        df_census_tracts = pd.read_csv("data/chicago_census_tracts.csv")
        df_census_tracts['the_geom'] = df_census_tracts['the_geom'].apply(shapely.wkt.loads)
        df_census_tracts = df_census_tracts.rename(columns={'the_geom': 'geometry'})
        df_census_tracts.head()
        df_census_tracts = df_census_tracts[["geometry", "GEOID10"]]

        gdf = gpd.GeoDataFrame(df_census_tracts, geometry='geometry')
        gdf = gdf.set_crs(epsg=4326, allow_override=True)
        gdf = gdf.to_crs(epsg=3857)

        buffer_distance = 500  # buffer distance in meters (500 m here)
        gdf_buffered = gdf.copy()
        gdf_buffered['geometry'] = gdf.buffer(buffer_distance)
        gdf['geometry'] = gdf_buffered.geometry  # Replace original geometry with buffered one
        census_tract_neighbors = gpd.sjoin(gdf, gdf, how='left', predicate='intersects')
        
        # Remove self-matches (where polygons intersect with themselves)
        census_tract_neighbors = census_tract_neighbors[census_tract_neighbors.index != census_tract_neighbors.index_right]
        census_tract_neighbors_grouped = census_tract_neighbors.groupby('GEOID10_left')['GEOID10_right'].apply(list)
        neighbors_dict = census_tract_neighbors_grouped.to_dict()
    


    def get_neighbor_demand_by_ID(df_timestamp, pickup_loc):
        neighbors = neighbors_dict[pickup_loc]
        neighbor_demand = df_timestamp[df_timestamp['pickup_loc'].isin(neighbors)]['demand_h-1'].mean()
        return neighbor_demand

    
    df['neighbor_demand_h-1'] = df.apply(lambda row: get_neighbor_demand_by_ID(df[df['timestamp'] == row['timestamp']], row['pickup_loc']), axis=1)
    return df

### Prepare dataset helpers

In [18]:
def get_prepared_data(location_type_hexagon=True, resolution=10):
    df = load_dataset_to_pandas(location_type_hexagon=location_type_hexagon, resolution=resolution)
    
    df = resample_to_hourly(df)
    df = merge_weather(df)
    df = get_past_temperature(df)
    df = get_past_precip(df)
    df = get_season(df)
    df = get_weekend(df)
    df = get_daytime(df)    
    df = get_holiday_dates(df)
    df = get_past_single_hour_demand(df)
    df = get_past_time_bucket_demand(df)    
    df = get_target_time_bucket_demand(df)    
    # df = get_neighbor_demand(df)
    
    return df

# Correlation Analysis


We have different measurement scales:

Ordinal (natural order, but no quantifiable difference between values or binary):
- season_x 
- trip_start_hour
- trip_start_month
- trip_start_day
Metric (equidistant scale):
- temp
- demand
- precipitation
- humidity
- wind_speed
- wind_gust
- pressure
Nominal:
- public_holiday
- weekend
- condition_Clear/Sunny etc.

We do 2 different analysis:
- Metric <-> Metric (Pearson)
- Ordinal & Nominal <-> Metric, Ordinal & Nominal <-> Ordinal & Nominal (Spearman)

In [19]:
metric = df_census[["demand", "demand_h-1", "demand_h-2", "demand_h-24", "temp", "temp_h-1", "temp_h-2", "temp_h-3", 'precip', 'precip_h-1', 'precip_h-2', 'precip_h-3', 'humidity', 'wind_speed', 'wind_gust', 'pressure']]
plt.figure(figsize=(12, 10))
sns.heatmap(metric.corr(method="pearson"), annot=True, cmap="RdYlGn")

NameError: name 'df_census' is not defined

In [20]:
ordinal_and_nominal = df_census[
        ["season_autumn", "season_spring", "season_summer", "season_winter", "daytime_morning_week",
        "daytime_afternoon_week", "daytime_evening_week", "daytime_night_week", "daytime_morning_weekend",
        "daytime_afternoon_weekend", "daytime_evening_weekend", "daytime_night_weekend", "public_holiday",
        "weekend"]] #"hour"
plt.figure(figsize=(20, 20))
sns.heatmap(pd.concat([ordinal_and_nominal, metric], axis=1).corr(method="spearman"), annot=True, cmap="RdYlGn")

# Prediction Models

## Preparation functions

### K-Fold Target encoding
With `pickup_loc` we still have a categorical feature in the dataset. As we we have more than 500 census tracts it does not make sense to do One hot encoding as we would obtain sparse data. Hence, we do k-fold target encoding with scikit-learn, which essentially uses a weighted sum of the mean of the target of each `pickup_loc` instead of the categorical `pickup_loc` value and the global mean. As this leads to leakage, we do this in combination with 5-fold cross fitting, as described under https://scikit-learn.org/stable/modules/preprocessing.html#target-encoder. This minimizes the impact of leakage and risk of overfitting. We fit the encoder only on the training data and apply the learnt transformation also to the test set, such that we have a consistent encoding.

In [34]:
def encode_pickup(df_train, df_test, target, encoding_name):
    encoder = TargetEncoder(target_type='continuous', cv=5, smooth='auto')
    encoder.fit(df_train['pickup_loc'].values.reshape(-1,1), df_train[target].values.reshape(-1,))
    df_train[encoding_name] = encoder.transform(df_train['pickup_loc'].values.reshape(-1,1))
    df_test[encoding_name] = encoder.transform(df_test['pickup_loc'].values.reshape(-1,1))
    return df_train, df_test

## Split the dataset
We split into 70, 15, 15

In [21]:
def train_test_split(df, train_ratio=0.8):
    def split_data(df, train_ratio):
        # First, we'll sort the dataframe just in case it isn't sorted
        df = df.dropna()
        df = df.sort_values('timestamp')
        
        # calculate the size of each section
        train_size = int(len(df) * train_ratio)
        
        # split the data
        train_df = df.iloc[:train_size]
        test_df = df.iloc[train_size:]
        
        return train_df, test_df

    # Apply the function for each 'pickup_loc' category
    train_dfs = []
    test_dfs = []

    for pickup_loc in df['pickup_loc'].unique():
        df_loc = df[df['pickup_loc'] == pickup_loc]
        train_df_loc, test_df_loc = split_data(df_loc, train_ratio)
        
        train_dfs.append(train_df_loc)
        test_dfs.append(test_df_loc)

    # Concatenate the dataframes
    train_df = pd.concat(train_dfs)
    test_df = pd.concat(test_dfs)
    
    return train_df, test_df


## Export Datasets

Now we export the dataset, such that we can use it 

In [24]:
# # For hexagons
# for resolution in list(range(3, 10)):
#     print(f"Processing resolution: {resolution}")
#     df = get_prepared_data(location_type_hexagon=True, resolution=resolution)
#     train_df, test_df = train_test_split(df)
    
#     for target in ['demand_target_bucket_1', 'demand_target_bucket_2', 'demand_target_bucket_6', 'demand_target_bucket_24']:
#         train_df, test_df = encode_pickup(train_df, test_df, target, f'pickup_{target}')
#     train_df.to_csv(f'./data/model_input/{resolution}_res_train.csv', index=False)
#     test_df.to_csv(f'./data/model_input/{resolution}_res_test.csv', index=False)
#     print(f"Resolution {resolution} data saved.")

# Now for census tract
print("Processing census tract data.")
df = get_prepared_data(location_type_hexagon=False)
train_df, test_df = train_test_split(df)
for target in ['demand_target_bucket_1', 'demand_target_bucket_2', 'demand_target_bucket_6', 'demand_target_bucket_24']:
    train_df, test_df = encode_pickup(train_df, test_df, target, f'pickup_{target}')
train_df.to_csv(f'./data/model_input/0_res_train.csv', index=False)
test_df.to_csv(f'./data/model_input/0_res_test.csv', index=False)
print("Census tract data saved.")


Processing census tract data.
Census tract data saved.
